In [14]:
import os
import numpy as np
import h5py
import dask.array as da

1) Создать массив размерностью 1000 на n (n - кратно тысячам), заполненный нормально распределенными числами (матожидание 0.0, стандартное отклонение 1.0), и сохранить его в файле(ах) с расширением hdf5

In [15]:
n = int(input("Input n (thousands): "))
x = da.random.standard_normal(size=(1000, 1000*n))
x

Input n (thousands): 4


dask.array<standard_normal, shape=(1000, 4000), dtype=float64, chunksize=(1000, 4000), chunktype=numpy.ndarray>

In [16]:
with h5py.File('random.hdf5', 'w') as f:
    dset = f.create_dataset("default", data=x)

In [17]:
%time da.random.standard_normal(size=(1000, 1000*n))

CPU times: user 1.46 ms, sys: 0 ns, total: 1.46 ms
Wall time: 1.43 ms


dask.array<standard_normal, shape=(1000, 4000), dtype=float64, chunksize=(1000, 4000), chunktype=numpy.ndarray>

In [18]:
%time np.random.normal(0, 1, size=(1000, 1000*n))

CPU times: user 186 ms, sys: 0 ns, total: 186 ms
Wall time: 189 ms


array([[-0.11043008,  2.0864264 , -0.12817487, ..., -0.5222898 ,
         0.21238651,  0.06202435],
       [ 1.63747136, -0.34084155,  0.02835588, ..., -0.60859933,
         1.46366976, -0.6654191 ],
       [ 0.99161944, -1.00621114,  0.24907748, ...,  1.17954167,
         0.73472157, -0.73127931],
       ...,
       [-2.06358806,  0.60260286,  1.72882418, ...,  0.75508791,
        -0.1597473 ,  0.43613201],
       [ 1.30497444, -0.91890716,  1.0179618 , ..., -1.37704871,
         1.08004715, -0.44327769],
       [ 0.57152431,  0.26369104, -0.41563802, ..., -1.44622785,
         0.08880012, -0.11376216]])

Решить 2я способами: с помощью NumPy и с помощью Dask Array и сравнить время выполнения:

2.1) Считать массив из файла и загрузить в dask array, и преобразовать отдельно в numpy array

2.2) Найти и вывести на экран наименьший элемент в массиве.

In [19]:
with h5py.File('random.hdf5', 'r') as f:
   data = f['default']
   a = da.from_array(data, chunks=(1000, 1000), lock=True)
   print(a)
   print(type(a))
   print(a.min().compute())

dask.array<array, shape=(1000, 4000), dtype=float64, chunksize=(1000, 1000), chunktype=numpy.ndarray>
<class 'dask.array.core.Array'>
-5.494124435440692


3) Сгенирировать нормально распределенный одномерный массив из 1000 элементов (вектор $a$). Найти ближайший к нему (в смысле $L2$) вектор-столбец из исходного массива (вектор $v_n$).

In [20]:
from scipy.spatial import distance
a = da.random.standard_normal(size=(1, 1000))
a

dask.array<standard_normal, shape=(1, 1000), dtype=float64, chunksize=(1, 1000), chunktype=numpy.ndarray>

In [21]:
from sklearn.neighbors import KDTree

In [22]:
a.compute()

array([[-1.67339995e+00,  1.96556506e-01, -1.02562376e-01,
         1.54247794e+00, -1.30065536e+00, -1.08602285e+00,
         6.77167735e-01, -5.72704891e-01, -4.99283179e-01,
        -7.62387032e-01,  6.27970198e-01, -1.55289834e+00,
        -2.36544832e+00, -3.25487664e-01,  7.17801599e-01,
        -4.51611060e-01,  4.98683257e-01,  1.27474400e+00,
         1.35102618e+00,  5.92726442e-01, -1.77608069e+00,
        -6.62127608e-01, -1.93729814e+00,  5.23303221e-01,
        -1.63366822e+00, -1.84208388e+00, -8.17357963e-01,
        -3.43594463e-01, -1.01898895e+00, -2.61936025e-01,
         1.18726174e+00,  1.07926415e+00,  1.55601160e+00,
         1.36930770e-01,  8.35696524e-01,  2.32018613e+00,
         9.31385643e-01, -6.33134670e-01,  3.62529016e-01,
         2.91533831e-01,  2.78369704e-01, -7.17580342e-01,
         5.72399027e-01, -1.40162075e-02,  5.91230921e-01,
        -4.24128809e-01,  1.22295223e+00,  5.67447015e-01,
         1.84168098e+00, -2.42712556e-01,  9.39631677e-0

In [23]:
tree = KDTree(a.reshape(-1, 1))
dist, i = tree.query(np.zeros(1000).reshape(-1, 1), k=1000)
L2 = dist[0] + a
L2.compute()

array([[-1.67182900e+00,  1.98516021e-01, -9.99508913e-02,
         1.54610140e+00, -1.29564844e+00, -1.08051291e+00,
         6.83420738e-01, -5.66116579e-01, -4.91793219e-01,
        -7.54328343e-01,  6.37100566e-01, -1.54319192e+00,
        -2.35530234e+00, -3.14749723e-01,  7.29701402e-01,
        -4.38626224e-01,  5.12328839e-01,  1.28876021e+00,
         1.36541546e+00,  6.07295277e-01, -1.76108429e+00,
        -6.45596485e-01, -1.91960394e+00,  5.41869969e-01,
        -1.61453176e+00, -1.82195430e+00, -7.93210013e-01,
        -3.19345284e-01, -9.94381424e-01, -2.35110409e-01,
         1.21494773e+00,  1.10799901e+00,  1.58696167e+00,
         1.71533896e-01,  8.71748828e-01,  2.35729766e+00,
         9.68633535e-01, -5.95866711e-01,  4.00480055e-01,
         3.33239315e-01,  3.20732612e-01, -6.71563236e-01,
         6.19025560e-01,  3.34118282e-02,  6.39657001e-01,
        -3.74415569e-01,  1.27310939e+00,  6.17661827e-01,
         1.89442088e+00, -1.89602508e-01,  9.94315300e-0

4) Определить сколько векторов-столбцов из исходного массива находятся на расстоянии, не превышающем $3|a-v_n|_{L2}$

In [24]:
dist = dist[0]

In [25]:
dis = 3*np.abs(L2 - a)
dist[dist<dis].shape[0] 

<ipython-input-25-a5c8a0f7094b>:2: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dist[dist<dis].shape[0]


1000